In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [152]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree
from math import atan2, degrees
import cv2
import numpy as np
from scipy import stats
img = cv2.imread('000196.jpg')
class Polygon_Proj:
    def __init__(self,file, img):
        self.root = object()
        self.data = file
        self.point_list = []
        self.img = img
        self.test = img.copy()
        
        self.useful = True
        self.removed_points = []
        
        self.c_figure = 0.3
        self.i = 0
        self.pts = []
    def get_xml(self):
        doc = ET.parse(self.data)
        self.root = doc.getroot()
        for labels in self.root[1][0]:
            if labels.tag == 'labels':
                label_name = [label[0].text for label in labels]
                break
        print('labels :', str(label_name).replace(",",''))
    
    def annotation(self):
        self.img = cv2.resize(self.img, dsize = (0,0), fx = 0.35, fy = 0.35, interpolation = cv2.INTER_LINEAR)
        self.test = cv2.resize(self.test, dsize = (0,0), fx = 0.35, fy = 0.35, interpolation = cv2.INTER_LINEAR)
        self.img = cv2.addWeighted(self.img, 0.7, self.test, 1, 3)
        cv2.imshow('image',self.img)
        k = cv2.waitKey()
        cv2.destroyAllWindows()
        
    def do_pro_coarse(self):
        distance = []
        for i in range(len(pts)-1):
            p1 = np.array(pts[i])
            p2= np.array(pts[i+1])
            distance.append(np.linalg.norm(p1-p2))
        median_distance = np.median(distance)
        removed_points = []
        
        return pts
    
    def check_tiny_polygon(self):
        distance = []
        for i in range(len(self.pts)-1):
            p1 = np.array(self.pts[i])
            p2= np.array(self.pts[i+1])
            distance.append(np.linalg.norm(p1-p2))
        median_distance = np.median(distance)
        removed_points = []
        #print(self.i,':',sum(distance))
        if sum(distance) < 150:
            self.useful = False
            return False
        else:
            self.useful = True
            return True
        self.i += 1
    
    def angle_between(self, points):
        x1, y1 = points[0]
        x2, y2 = points[1]
        x3, y3 = points[2]
        deg1 = (360 + degrees(atan2(x1 - x2, y1 - y2))) % 360
        deg2 = (360 + degrees(atan2(x3 - x2, y3 - y2))) % 360
        return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)
    
    def remove_small_degree_points(self):
        removed_points = []
        for i in range(len(self.pts)):
            if i > len(self.pts) - 3:
                break
            degree = self.angle_between(self.pts[i:i+3])
            if degree < 50:
                removed_points.append(i+1)
        #self.removed_points += removed_points
        self.pts = [p for i, p in enumerate(self.pts) if i not in removed_points]
        if bool(removed_points) == False:
            return
        else:
            self.remove_small_degree_points()
    def point_test(self, min_point, max_point):
        #print(self.pts)
        #pts = [p for i, p in enumerate(pts) if i not in self.removed_points]
        self.removed_points = []
        x_y_ratio = 1 # default ratio of x and y
        # make the rectangle using min, max point
        x_axis = max_point[0] - min_point[0]
        y_axis = max_point[1] - min_point[1]
        
        # get the ratio of x, y because of reducing same size with x and y
        # define how many size should be reduced.
        if x_axis > y_axis:
            x_y_ratio = x_axis/y_axis
            x_size = (x_axis * self.c_figure)/x_y_ratio
            y_size = y_axis * self.c_figure
        else:
            x_y_ratio = y_axis/x_axis
            x_size = x_axis * self.c_figure
            y_size = (y_axis * self.c_figure)/x_y_ratio
        
        # relative point list for each point x, y
        rel_point_x = []
        rel_point_y = []
        
        # get relative points
        for p in self.pts:
            dist_x = p[0] - min_point[0]  # get distance from start to x point
            dist_y = p[1] - min_point[1] # get distance from start to y point
            rel_point_x.append(dist_x/x_axis)
            rel_point_y.append(dist_y/y_axis)
            
        # reduce the total size
        #define the position min_x, min_y for reducing size
        #x와 y 사이 길이를 줄일 때, 줄어든 만큼의 길이를 둘로 나누어 줄어든 후 x,y의 위치를 결정
        # x, y 사이 길이가 10에서 8로 줄어들면, x,y위치는 각각 x + 1, y - 1이 된다. 여기서 1을 구하기 위해
        #10 - 8 / 2 = 1
        x_axis -= x_size
        y_axis -= y_size
        moved_x = (x_size)/2
        moved_y = (y_size)/2
        # 시작위치를 정하고 줄어든 만큼의 거리에 해당 포인트들을 옮기기
        
        for p, x, y in zip(self.pts, rel_point_x, rel_point_y):
            p[0] = (x * x_axis) + (min_point[0] + moved_x)
            p[1] = (y * y_axis) + (min_point[1] + moved_y)
        
    def do_basic_coarse(self):
        max_point = np.max(self.pts,axis=0)
        min_point = np.min(self.pts, axis=0)
        #if self.count == 10:
            #self.test_pts(pts, min_point, max_point)
        if self.check_tiny_polygon():
            self.remove_small_degree_points()
            self.point_test(min_point, max_point)
        '''
        for p in pts:
            if p[0] == max_point[0] and p[1] == max_point[1]:
                continue
            elif p[0] == max_point[0] and p[1] == min_point[1]:
                continue
            elif p[0] == min_point[0] and p[1] == min_point[1]:
                continue
            elif p[0] == min_point[0] and p[1] == max_point[1]:
                continue

            if p[0] >= mean_point[0]:
                p[0] = p[0] * (1 - self.c_figure)
            else:
                p[0] = p[0] * (1 + self.c_figure)
            if p[1] >= mean_point[1]:
                p[1] =  p[1] * (1 - self.c_figure)
            else:
                p[1] = p[1] * (1 + self.c_figure)
        '''

    def prune_points(self,sub_root):
        for polygon in sub_root: 
            self.point_list.append(polygon.attrib['points'])
        
        for points in self.point_list:
            points = points.split(';')
            for point in points:
                temp = point.split(',') # extract x, y
                self.pts.append(list(map(float, temp))) # chaining x, y
            self.do_basic_coarse()
            self.pts = np.array(self.pts, np.float32)
            self.pts = self.pts.astype(int)
            self.pts = self.pts.reshape((-1,1,2))
            #if self.count == 10:
            if self.useful == True:
                color = list(map(int,(np.random.randint(256,size=3))))
                self.img = cv2.fillPoly(self.img, [self.pts], color)
                self.test = cv2.polylines(self.test, [self.pts], True, color, 3)
                
            self.pts = []
                 
        self.annotation()
    def parse_tree(self):
        for sub_root in self.root:
            if sub_root.tag == 'image' and sub_root.attrib['name'] == '000196.jpg':
                self.prune_points(sub_root)
                break
            else:
                continue
        #tree = ElementTree(self.root)
        #tree.write('ssuu.xml')
if __name__ == '__main__':
    polygon = Polygon_Proj('ssu_03.xml',img)
    polygon.get_xml()
    polygon.parse_tree()
    print("done")
   

labels : ['vehicle' 'human' 'building' 'river' 'mountain' 'sky' 'etc' 'guard_rail' 'poll' 'bush' 'forest' 'tree' 'grass' 'cobbled_road' 'dirt_road' 'packing' 'line' 'asphalt']
done


In [151]:
https://bkshin.tistory.com/entry/OpenCV-9-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%97%B0%EC%82%B0

SyntaxError: invalid syntax (Temp/ipykernel_19904/3647522683.py, line 1)

In [6]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import Frame, filedialog, messagebox
from tkinter.filedialog import askopenfilenames
from PIL import Image
from PIL import ImageTk
from tkinter.ttk import Progressbar
from math import atan2, degrees
import itertools
import threading
import time
import sys

root = tk.Tk()

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree


class Polygon_Proj:
    def __init__(self,root):
        self.root = object()
        self.data = object()
        self.label1 = object()  # fine image object for next, prev
        self.label2 = object()  # coarse image object for next, prev
        
        self.window = root
        
        self.total_coarse = []
        self.total_mask = []
        
        self.coarse_list = []  # list of coarse level
        self.mask_list = []  # list of mask level
        self.annotation_color = {}
        
        self.img_dict = {}
        self.raw_img_list = {}
        self.fine_img_list = {}
        self.coarse_img_list = {}
        
        self.masked_fine_img = {}
        self.masked_coarse_img = {}
        self.index = 0 # order of image
        
        self.c_figure = 0.1  # coarse figure
        self.useful = True  # check size of polygon
        self.point_list = []  # annotation point list
        self.f_option = 1   # choose the level of coarse
        self.option = 0
        self.degree = 30 # to remove sharp point
        
    def upload_img(self):
        image_formats = [("JPEG","*.jpg")]
        file_path_list = askopenfilenames(filetypes=image_formats, initialdir="/", title='Please select a picture to analyze')

        for file in file_path_list:
            img_name = file.split('/')
            img_name = img_name.pop()
            img = cv2.imread(file, cv2.IMREAD_COLOR)
            self.raw_img_list[img_name] = img
        tk.messagebox.showinfo('image upload','image uploaded!')
        
    def upload_xml(self):
        self.data = filedialog.askopenfilename(initialdir="/", title="Select file",
                                          filetypes=(("xml files", "*.xml"),
                                          ("all files", "*.*")))
        doc = ET.parse(self.data)
        self.root = doc.getroot()
        for labels in self.root[1][0]:
            if labels.tag == 'labels':
                label_name = [label[0].text for label in labels]
                break
        print('labels :', str(label_name).replace(",",''))
        tk.messagebox.showinfo('xml upload','xml uploaded!')

    
            
    def xml_warning(self):
        if messagebox.askokcancel("warning!", "wrong xml file"):
            pass
        else:
            self.window.destroy()
            
    def main_work(self):
        start_time = time.time()
        for img in self.img_dict.keys():
            self.draw_annotation(1, img, self.img_dict[img])
        while self.degree < 70:
            while self.c_figure < 0.5:
                for img in self.img_dict.keys():
                    self.draw_annotation(2, img, self.img_dict[img])
                self.coarse_list.append(self.coarse_img_list)
                self.mask_list.append(self.masked_coarse_img)
                self.c_figure += 0.1
                self.coarse_img_list = {}
                self.masked_coarse_img = {}
            self.total_coarse.append(self.coarse_list)
            self.total_mask.append(self.mask_list)
            self.coarse_list = []
            self.mask_list = []
            self.degree += 10
            self.c_figure = 0.1
        print("---{}s seconds---".format(time.time()-start_time))
        self.degree = 30
        self.open_img()
        
    def extract_points(self,sub_root, img_name):
        point_list = []
        for points in sub_root: 
            polygon = points.attrib
            point_list.append(polygon['points'])
        self.img_dict[img_name] = point_list
        #self.draw_annotation(1, img_name, point_list)
        #self.draw_annotation(2, img_name, point_list)
    def parse_tree(self):
        for sub_root in self.root:
            if sub_root.tag == 'image':
                if sub_root.attrib['name'] in self.raw_img_list.keys():
                    self.extract_points(sub_root, sub_root.attrib['name'])
                else:
                    self.xml_warning()
                    break
        self.main_work()
        
        #tk.messagebox.showinfo('annotation','annotation finished!')
    def show_cv(self, img, cv2):
        img = cv2.resize(img, dsize = (0,0), fx = 0.3, fy = 0.3, interpolation = cv2.INTER_LINEAR)
        cv2.imshow('image',img)
        k = cv2.waitKey()
        cv2.destroyAllWindows()
        
    def angle_between_three_points(self, points):
        x1, y1 = points[0]
        x2, y2 = points[1]
        x3, y3 = points[2]
        deg1 = (360 + degrees(atan2(x1 - x2, y1 - y2))) % 360
        deg2 = (360 + degrees(atan2(x3 - x2, y3 - y2))) % 360
        return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)
    
    def remove_small_degree_points(self):
        removed_points = []
        for i in range(len(self.point_list)):
            if i > len(self.point_list) - 3:
                break
            degree = self.angle_between_three_points(self.point_list[i:i+3])
            if degree < self.degree:
                removed_points.append(i+1)
        #self.removed_points += removed_points
        self.point_list = [p for i, p in enumerate(self.point_list) if i not in removed_points]
        if bool(removed_points) == False:
            return
        else:
            self.remove_small_degree_points()
            
    def check_tiny_polygon(self):
        distance = []
        for i in range(len(self.point_list)-1):
            p1 = np.array(self.point_list[i])
            p2= np.array(self.point_list[i+1])
            distance.append(np.linalg.norm(p1-p2))
        if sum(distance) < 200: # remove tiny polygon from annotation
            return False
        else:
            return True
            
    def reduce_annotation(self, min_point, max_point):
        x_y_ratio = 1 # default ratio of x and y
        # make the rectangle using min, max point
        x_axis = max_point[0] - min_point[0]
        y_axis = max_point[1] - min_point[1]
        
        # get the ratio of x, y because of reducing same size with x and y
        # define how many size should be reduced.
        if x_axis > y_axis:
            x_y_ratio = x_axis/y_axis
            x_size = (x_axis * self.c_figure)/x_y_ratio
            y_size = y_axis * self.c_figure
        else:
            x_y_ratio = y_axis/x_axis
            x_size = x_axis * self.c_figure
            y_size = (y_axis * self.c_figure)/x_y_ratio
        
        # relative point list for each point x, y
        rel_point_x = []
        rel_point_y = []
        
        # get relative points
        for p in self.point_list:
            dist_x = p[0] - min_point[0]  # get distance from start to x point
            dist_y = p[1] - min_point[1] # get distance from start to y point
            rel_point_x.append(dist_x/x_axis)
            rel_point_y.append(dist_y/y_axis)
            
        # reduce the total size
        #define the position min_x, min_y for reducing size
        #x와 y 사이 길이를 줄일 때, 줄어든 만큼의 길이를 둘로 나누어 줄어든 후 x,y의 위치를 결정
        # x, y 사이 길이가 10에서 8로 줄어들면, x,y위치는 각각 x + 1, y - 1이 된다. 여기서 1을 구하기 위해
        #10 - 8 / 2 = 1
        x_axis -= x_size
        y_axis -= y_size
        moved_x = (x_size)/2
        moved_y = (y_size)/2
        # 시작위치를 정하고 줄어든 만큼의 거리에 해당 포인트들을 옮기기
        
        for p, x, y in zip(self.point_list, rel_point_x, rel_point_y):
            p[0] = (x * x_axis) + (min_point[0] + moved_x)
            p[1] = (y * y_axis) + (min_point[1] + moved_y)
            
    def do_coarse(self):
        max_point = np.max(self.point_list,axis=0)
        min_point = np.min(self.point_list, axis=0)
        
        self.reduce_annotation(min_point, max_point)
        self.remove_small_degree_points()
    
    def draw_annotation(self, option, img_name, point_list):
        if option == 1:
            annotation_color = []
            img1 = self.raw_img_list[img_name].copy()
            masked_img = self.raw_img_list[img_name].copy()
            for points in point_list:
                points = points.split(';')
                for point in points:
                    temp = point.split(',')
                    self.point_list.append(list(map(float, temp)))
                self.point_list = np.array(self.point_list, np.int32)
                self.point_list = self.point_list.reshape((-1,1,2))
                color = list(map(int,(np.random.randint(256,size=3))))
                annotation_color.append(color)
                masked_img = cv2.fillPoly(masked_img, [self.point_list], color)
                img1 = cv2.polylines(img1, [self.point_list], True, color, 3)
                self.point_list = []
            self.annotation_color[img_name] = annotation_color
            img1 = cv2.resize(img1, dsize = (0,0), fx = 0.33, fy = 0.33, interpolation = cv2.INTER_AREA)
            masked_img = cv2.resize(masked_img, dsize = (0,0), fx = 0.33, fy = 0.33, interpolation = cv2.INTER_AREA)
            masked_img = cv2.addWeighted(masked_img, 0.7, img1, 1, 3)
            self.fine_img_list[img_name] = img1
            self.masked_fine_img[img_name] = masked_img
        else:
            img2 = self.raw_img_list[img_name].copy()
            masked_img = self.raw_img_list[img_name].copy()
            color = self.annotation_color[img_name]
            for i, points in enumerate(point_list):
                points = points.split(';')
                for point in points:
                    temp = point.split(',')
                    self.point_list.append(list(map(float, temp)))
                if self.check_tiny_polygon():
                    self.do_coarse()
                    self.point_list = np.array(self.point_list, np.float32)
                    self.point_list = self.point_list.astype(int)
                    self.point_list = self.point_list.reshape((-1,1,2))
                    masked_img = cv2.fillPoly(masked_img, [self.point_list], color[i])
                    img2 = cv2.polylines(img2, [self.point_list], True, color[i], 3)
                self.point_list = []
            img2 = cv2.resize(img2, dsize = (0,0), fx = 0.33, fy = 0.33, interpolation = cv2.INTER_AREA)
            masked_img = cv2.resize(masked_img, dsize = (0,0), fx = 0.33, fy = 0.33, interpolation = cv2.INTER_AREA)
            masked_img = cv2.addWeighted(masked_img, 0.7, img2, 1, 0)
            self.coarse_img_list[img_name] = img2
            self.masked_coarse_img[img_name] = masked_img
            
            
    def paint_polygon(self):
        keys = list(self.fine_img_list.keys())
        img1 = cv2.cvtColor(self.masked_fine_img[keys[self.index]], cv2.COLOR_BGR2RGB)
        img1 = Image.fromarray(img1)
        img2 = cv2.cvtColor(self.total_mask[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
        img2 = Image.fromarray(img2)

        imgtk1 = ImageTk.PhotoImage(image=img1)
        imgtk2 = ImageTk.PhotoImage(image=img2)

        self.label1.config(image=imgtk1)
        self.label1.image = imgtk1

        self.label2.config(image=imgtk2)
        self.label2.image = imgtk2
    

    # change degree to make smoothie polygon    
    def change_shape(self):
        if self.option == 3:
            self.option = 0
        else:
            self.option += 1

        keys = list(self.fine_img_list.keys())

        img2 = cv2.cvtColor(self.total_coarse[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
        img2 = Image.fromarray(img2)
        imgtk2 = ImageTk.PhotoImage(image=img2)

        self.label2.config(image=imgtk2)
        self.label2.image = imgtk2
        
        degree = tk.Label(self.window, text = str(self.degree + self.option * 10)).place(x= 460, y = 10)
        #self.paint_polygon()
    
    def increase_figure(self):
        if self.f_option >= 3:
            pass
        else:
            self.f_option += 1
            keys = list(self.fine_img_list.keys())

            img2 = cv2.cvtColor(self.total_coarse[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
            img2 = Image.fromarray(img2)
            imgtk2 = ImageTk.PhotoImage(image=img2)

            self.label2.config(image=imgtk2)
            self.label2.image = imgtk2
            self.paint_polygon()
            
    def decrease_figure(self):
        if self.f_option <= 0:
            pass
        else:
            self.f_option -= 1
            keys = list(self.fine_img_list.keys())

            img2 = cv2.cvtColor(self.total_coarse[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
            img2 = Image.fromarray(img2)
            imgtk2 = ImageTk.PhotoImage(image=img2)

            self.label2.config(image=imgtk2)
            self.label2.image = imgtk2
            self.paint_polygon()
            
    def prev_next_warning(self,choice):
        if choice == 1:
            if messagebox.askokcancel("warning!", "no prev image."):
                pass
            else:
                pass
        else:
            if messagebox.askokcancel("warning!", "no next image."):
                pass
            else:
                pass
    
    def next_img(self):
        if self.index >= len(self.raw_img_list)-1:
            self.prev_next_warning(2)
        else:
            self.index += 1
            keys = list(self.fine_img_list.keys())
            img1 = cv2.cvtColor(self.fine_img_list[keys[self.index]], cv2.COLOR_BGR2RGB)
            img1 = Image.fromarray(img1)
            img2 = cv2.cvtColor(self.total_coarse[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
            img2 = Image.fromarray(img2)

            imgtk1 = ImageTk.PhotoImage(image=img1)
            imgtk2 = ImageTk.PhotoImage(image=img2)

            self.label1.config(image=imgtk1)
            self.label1.image = imgtk1

            self.label2.config(image=imgtk2)
            self.label2.image = imgtk2

    def prev_img(self):
        if self.index <= 0:
            self.prev_next_warning(1)
        else:
            self.index -= 1
            keys = list(self.fine_img_list.keys())
            img1 = cv2.cvtColor(self.fine_img_list[keys[self.index]], cv2.COLOR_BGR2RGB)
            img1 = Image.fromarray(img1)
            img2 = cv2.cvtColor(self.total_coarse[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
            img2 = Image.fromarray(img2)

            imgtk1 = ImageTk.PhotoImage(image=img1)
            imgtk2 = ImageTk.PhotoImage(image=img2)

            self.label1.config(image=imgtk1)
            self.label1.image = imgtk1

            self.label2.config(image=imgtk2)
            self.label2.image = imgtk2   
    def open_img(self):
        keys = list(self.fine_img_list.keys())
        img1 = cv2.cvtColor(self.fine_img_list[keys[self.index]], cv2.COLOR_BGR2RGB)
        img1 = Image.fromarray(img1)
        img2 = cv2.cvtColor(self.total_coarse[self.option][self.f_option][keys[self.index]], cv2.COLOR_BGR2RGB)
        img2 = Image.fromarray(img2)
        
        imgtk1 = ImageTk.PhotoImage(image=img1)
        imgtk2 = ImageTk.PhotoImage(image=img2)

        self.label1 = tk.Label(self.window, image=imgtk1)
        self.label1.image = imgtk1
        #self.label1.place (x = 12, y = 45)
        self.label1.pack(side = 'top', anchor = 'n', expand = 'yes', fill = 'both')
        
        self.label2 = tk.Label(self.window, image=imgtk2)
        self.label2.image = imgtk2   #class 내에서 작업할 경우에는 이 부분을 넣어야 보인다.
        #self.label2.place(x = 12, y = 430)
        self.label2.pack(side = 'top', anchor = 'n', expand = 'yes', fill = 'both',pady = 10)
    def download(self):
        pass
        #tree = ElementTree(self.root)
        #tree.write('ssu.xml')
    def main_func(self):
        
        self.window.title("script window")
        self.window.geometry("660x790+100+100")
        #self.window.resizable(False, False)
        frame_one = Frame(self.window, width = 655, height = 37)
        frame_one['borderwidth'] = 2
        frame_one['relief'] = 'sunken'
        #frame_one.place(x = 10, y = 5)
        frame_one.pack(side='top',anchor = 'nw',expand = 'yes',fill = 'x', padx = 5, ipady = 1)
        
        img_button = tk.Button(self.window, text = 'upload image', bg = '#e0e0e0', fg = 'black',
                          font = ('sans 10'), command = self.upload_img).place(x = 10, y = 8)
        xml_button = tk.Button(self.window, text = 'upload xml', bg = '#e0e0e0', fg = 'black',
                          font = ('sans 10'), command = self.upload_xml).place(x = 117, y = 8)
        annotation_button = tk.Button(self.window, text = 'start annotation', bg = '#e0e0e0', fg = 'black',
                          font = ('sans 10'), command = self.parse_tree).place(x = 210, y = 8)
        degree_button = tk.Button(self.window, text = 'degree', bg = '#e0e0e0', fg = 'black',
                          font = ('sans 10'), command = self.change_shape).place(x = 390, y = 8)
        dcrease_button = tk.Button(self.window, text = '-', bg = '#e0e0e0', fg = 'black',
                          font = ('sans 10'), command = self.increase_figure).place(x = 330, y = 8)
        increase_button = tk.Button(self.window, text = '+', bg = '#e0e0e0', fg = 'black',
                          font = ('sans 10'), command = self.decrease_figure).place(x = 360, y = 8)
        degree = tk.Label(self.window, text = str(self.degree + self.option * 10)).place(x= 460, y = 10)
        
        prev_button = tk.Button(self.window, text = 'prev', bg = '#bdbdbd', fg = 'black',
                          font = ('sans 10'), command = self.prev_img).place(x = 507, y = 8)
        next_button = tk.Button(self.window, text = 'next', bg = '#bdbdbd', fg = 'black',
                          font = ('sans 10'), command = self.next_img).place(x = 557, y = 8)
        mask_button = tk.Button(self.window, text = 'mask', bg = '#bdbdbd', fg = 'black',
                          font = ('sans 10'), command = self.paint_polygon).place(x = 605, y = 8)
    
        self.window.mainloop()
        
if __name__ == '__main__':
    polygon = Polygon_Proj(root)
    polygon.main_func()
    print("done")

done


In [32]:
import numpy as np
import math
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression

a = np.array([5,5])
b = np.array([10,5])
c = np.array([1,4,7,9,10,100,200])
c = np.delete(c,[1,3,5])
aa = [1,4,7,9,10,100,200]

def xx(self,pts):
        x = []
        y = []
        for p in pts:
            x.append(p[0])
            y.append(p[1])
        x = np.array(x).reshape(-1,1)
        fit = LinearRegression().fit(x,y)
        residual = y - fit.predict(x)
        print(residual)

import itertools
import threading
import time
import sys

done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/','-','\\']):
        if done:
            break
        sys.stdout.write('\rloading ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\rDone!     ')

t = threading.Thread(target=animate)
t.start()
for i in range(100000000):
    t = i + 10
#long process here
#time.sleep(10)
done = True

Done!     